In [90]:
# first run installations and data download
! pip install -U llama-index llama-index-vector-stores-milvus pymilvus llama-index-llms-openai llama-index-readers-file
! mkdir -p './storage/commits'


zsh:1: command not found: pip


In [91]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.core.tools import QueryEngineTool, ToolMetadata

from pymilvus import connections
from llama_index.vector_stores.milvus import MilvusVectorStore

In [92]:
try:
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/commits"
    )
    commit_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

In [93]:
from dotenv import load_dotenv
import os
load_dotenv()

True

# Build the Milvus Index from a Git Patch
1. To generate a patch, in terminal type _git format-patch -1_
2. Ingest the patch into Milvus

In [94]:
if not index_loaded:
    # load data
    commit_docs = SimpleDirectoryReader(
        input_files=["./0001-adding-a-better-calculator.patch"]
    ).load_data()

    # build index
    vector_store_commit = MilvusVectorStore(dim=1536, collection_name="commit", overwrite=True)
    
    storage_context_commit = StorageContext.from_defaults(vector_store=vector_store_commit)
    commit_index = VectorStoreIndex.from_documents(commit_docs, storage_context=storage_context_commit)


    # persist index
    commit_index.storage_context.persist(persist_dir="./storage/commit")

commit_engine = commit_index.as_query_engine(similarity_top_k=3)

# Using Llama Index build a summary of the Commit

In [96]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=commit_engine,
        metadata=ToolMetadata(
            name="commit_summary",
            description=(
                "Provides summary information about a commit patch.  "
                # "Need to manually run 'git format-patch -1 to generate the patch file."
            ),
        ),
    ),
]

In [97]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI

In [98]:
llm = OpenAI(model="gpt-3.5-turbo-0613")

agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
    # context=context
)

# Create 3 Llama Index Agents - Title, Summary, Body

In [99]:
agent1_response = agent.chat("Describe in great detail what changes were made in this patch?")
print(str(agent1_response))

Thought: I need to use a tool to help me answer the question.
Action: commit_summary
Action Input: {'input': 'describe the changes in this patch'}
Observation: The changes in this patch include the addition of a new Jupyter notebook file named "better_calculator_agent.ipynb" with 212 insertions. This notebook contains code cells that define functions for mathematical operations like multiplication, addition, and squaring. Additionally, it imports necessary modules and tools for these operations. The patch also updates the model used in the OpenAI instance to "gpt-4" and creates an agent with tools for multiplication, addition, and squaring operations. The notebook demonstrates step-by-step calculations for various mathematical expressions using these tools.
Thought: I can answer without using any more tools.
Answer: The changes in this patch include the addition of a new Jupyter notebook file named "better_calculator_agent.ipynb" with 212 insertions. This notebook contains code cells t

In [100]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI
from llama_index.core.tools import FunctionTool

In [127]:
def set_title(commit_body) -> str:
    """Takes in a commit body and returns a recommended blog title"""
    #submit commit_body to OpenAI's GPT-3.5 API
    openai = OpenAI()
    # generate a title from the commit body using open ai
    response = openai.completion_to_prompt("generate a blog title of 10 words or less for the following code changes : " + commit_body)
    return response


title_tool = FunctionTool.from_defaults(fn=set_title)
title_agent = ReActAgent.from_tools([title_tool], llm=llm, verbose=True)
response_gen = title_agent.chat(str(agent1_response))
print(str(response_gen))

Thought: I need to use a tool to help me generate a blog title based on the provided commit body.
Action: set_title
Action Input: {'commit_body': "The changes in this patch include the addition of a new Jupyter notebook file named 'better_calculator_agent.ipynb' with 212 insertions. This notebook contains code cells that define functions for mathematical operations like multiplication, addition, and squaring. Additionally, it imports necessary modules and tools for these operations. The patch also updates the model used in the OpenAI instance to 'gpt-4' and creates an agent with tools for multiplication, addition, and squaring operations. The notebook demonstrates step-by-step calculations for various mathematical expressions using these tools."}
Observation: generate a blog title of 10 words or less for the following code changes : The changes in this patch include the addition of a new Jupyter notebook file named 'better_calculator_agent.ipynb' with 212 insertions. This notebook cont

In [130]:
def set_summary(commit_body) -> str:
    """Takes in a commit body and returns a recommended commit summary"""
    #submit commit_body to OpenAI's GPT-3.5 API
    openai = OpenAI()
    # generate a title from the commit body using open ai
    response = openai.set_completion_to_prompt("Generate a brief summary. The summary shiould be 20 words.  " + commit_body)
    return response


summary_tool = FunctionTool.from_defaults(fn=set_summary)
summary_agent = ReActAgent.from_tools([summary_tool], llm=llm, verbose=True)
response_summary = summary_agent.chat(str(agent1_response))

Thought: I need to generate a commit summary for the changes described in the patch.
Action: set_summary
Action Input: {'commit_body': "The changes in this patch include the addition of a new Jupyter notebook file named 'better_calculator_agent.ipynb' with 212 insertions. This notebook contains code cells that define functions for mathematical operations like multiplication, addition, and squaring. Additionally, it imports necessary modules and tools for these operations. The patch also updates the model used in the OpenAI instance to 'gpt-4' and creates an agent with tools for multiplication, addition, and squaring operations. The notebook demonstrates step-by-step calculations for various mathematical expressions using these tools."}
Observation: Generate a brief summary. The summary shiould be 20 words.  The changes in this patch include the addition of a new Jupyter notebook file named 'better_calculator_agent.ipynb' with 212 insertions. This notebook contains code cells that defin

In [132]:
def set_body(commit_body) -> str:
    """Takes in a commit body and returns a blog post body describing the commit changes in great detail."""
    #submit commit_body to OpenAI's GPT-3.5 API
    openai = OpenAI()
    # generate a title from the commit body using open ai
    response = openai.set_completion_to_prompt("Generate a detailed blog post about the changes described in this commit. The summary must be written in a markdown and should be between 1000 to 2000 words. Include at least 2 Heading 2 headings.  " + commit_body)
    return response


body_tool = FunctionTool.from_defaults(fn=set_body)
body_agent = ReActAgent.from_tools([body_tool], llm=llm, verbose=True)
response_body = body_agent.chat(str(agent1_response))

Thought: I need to use a tool to help me summarize the changes in the patch.
Action: set_body
Action Input: {'commit_body': "The changes in this patch include the addition of a new Jupyter notebook file named 'better_calculator_agent.ipynb' with 212 insertions. This notebook contains code cells that define functions for mathematical operations like multiplication, addition, and squaring. Additionally, it imports necessary modules and tools for these operations. The patch also updates the model used in the OpenAI instance to 'gpt-4' and creates an agent with tools for multiplication, addition, and squaring operations. The notebook demonstrates step-by-step calculations for various mathematical expressions using these tools."}
Observation: Generate a detailed blog post about the changes described in this commit. The summary must be written in a markdown and should be between 1000 to 2000 words. Include at least 2 Heading 2 headings.  The changes in this patch include the addition of a ne

# Put it all together - Directive Approach

In [133]:
# blog_agent = ReActAgent.from_tools([summary_tool, title_tool], llm=llm, verbose=True)

def write_blog():
    title = title_agent.chat(str(agent1_response))
    summary = summary_agent.chat(str(agent1_response))
    body = body_agent.chat(str(agent1_response))
    
    save_note(str(title))
    save_note(str(summary))
    save_note(str(body))
    return title, summary, body

title, summary, body = write_blog()

Thought: I need to use a tool to help me answer the question.
Action: set_title
Action Input: {'commit_body': "The changes in this patch include the addition of a new Jupyter notebook file named 'better_calculator_agent.ipynb' with 212 insertions. This notebook contains code cells that define functions for mathematical operations like multiplication, addition, and squaring. Additionally, it imports necessary modules and tools for these operations. The patch also updates the model used in the OpenAI instance to 'gpt-4' and creates an agent with tools for multiplication, addition, and squaring operations. The notebook demonstrates step-by-step calculations for various mathematical expressions using these tools."}
Observation: generate a blog title of 10 words or less for the following code changes : The changes in this patch include the addition of a new Jupyter notebook file named 'better_calculator_agent.ipynb' with 212 insertions. This notebook contains code cells that define function

# Put it all together - Independent Agent Approach

In [135]:
overall_agent = ReActAgent.from_tools([title_tool, summary_tool, body_tool], llm=llm, verbose=True)
overall_summary = overall_agent.chat(str(agent1_response))

Thought: I can start by generating a title for the blog post based on the provided commit information.
Action: set_title
Action Input: {'commit_body': "The changes in this patch include the addition of a new Jupyter notebook file named 'better_calculator_agent.ipynb' with 212 insertions. This notebook contains code cells that define functions for mathematical operations like multiplication, addition, and squaring. Additionally, it imports necessary modules and tools for these operations. The patch also updates the model used in the OpenAI instance to 'gpt-4' and creates an agent with tools for multiplication, addition, and squaring operations. The notebook demonstrates step-by-step calculations for various mathematical expressions using these tools."}
Observation: generate a blog title of 10 words or less for the following code changes : The changes in this patch include the addition of a new Jupyter notebook file named 'better_calculator_agent.ipynb' with 212 insertions. This notebook

# STRETCH: Add a Tool that May not be used

In [145]:
def set_related_articles(related_articles) -> str:
    """If there are related articles provided, adds a link to the article and describe the articles. """
    #submit commit_body to OpenAI's GPT-3.5 API
    openai = OpenAI()
    # generate a title from the commit body using open ai
    response = openai.set_completion_to_prompt("Generate a likely description of the article based on the url, followed by the URL, for the following URLs.  " + str(related_articles))
    return response

related_articles = "https://www.acko.com/calculators/ advanced calculators article"

related_articles_tool = FunctionTool.from_defaults(fn=set_related_articles)
related_articles_agent = ReActAgent.from_tools([related_articles_tool], llm=llm, verbose=True)
response_related_articles = related_articles_agent.chat(related_articles)

overall_agent = ReActAgent.from_tools([title_tool, summary_tool, body_tool, related_articles_tool], llm=llm, verbose=True)
overall_summary = overall_agent.chat(str(agent1_response))

Thought: I need to extract information from the provided URL about how to import functions from another Jupyter notebook.
Action: set_related_articles
Action Input: {'related_articles': AttributedDict([('title', 'Related Articles')])}
Observation: Generate a likely description of the article based on the url, followed by the URL, for the following URLs.  AttributedDict([('title', 'Related Articles')])
Thought: I can generate a likely description of the article based on the URL provided.
Answer: Description: The article likely explains how to import functions from another Jupyter notebook.
URL: saturncloud.io/blog/how-to-import-functions-from-another-jupyter-notebook/
Thought: I can use the set_title tool to generate a recommended blog title based on the provided commit body.
Action: set_title
Action Input: {'commit_body': "The changes in this patch include the addition of a new Jupyter notebook file named 'better_calculator_agent.ipynb' with 212 insertions. This notebook contains code 